In [ ]:
import essentia as es
import essentia.standard as es_mode

Módulo com minhas funções de extrair e salvar _features_

In [ ]:
pk_extracted = "extracted"
pk_spectrum = pk_extracted+".spectrum"

pk_feat = "features"
pk_number = pk_feat+".number"
pk_target = pk_number+".target"
pk_energy_band_ratio = pk_number+".energy_band_ratio"
pk_max_mag_freq = pk_number+".max_mag_freq"
pk_spectral_centroid_time = pk_number+".spectral_centroid_time"
pk_spectral_complexity = pk_number+".spectral_complexity"
pk_strong_peak = pk_number+".strong_peak"
pk_inharmonicity = pk_number+".inharmonicity"

pk_array = pk_feat+".array"
pk_spectral_contrast = pk_array+".spectral_contrast"
pk_spectral_valley = pk_array+".spectral_valley"
pk_spectral_peaks_freq = pk_array+".spectral_peaks_freq"
pk_spectral_peaks_mag = pk_array+".spectral_peaks_mag"
pk_spectral_whitening = pk_array+".spectral_whitening"

#Auxiliares
pk_auxiliar = "auxiliar"
pk_pitch = pk_auxiliar+".pitch"
pk_pitch_prob = pk_auxiliar+".pitch_prob"
pk_harmonic_peaks_freq = pk_auxiliar+".harmonic_peaks_freq"
pk_harmonic_peaks_mag = pk_auxiliar+".harmonic_peaks_mag"

def printListNewline(input_list):
  """"
  Printa input_list de forma mais ordenada
  """
  print("[\n ", ",\n  ".join(input_list),"\n]")

def printPoolDescriptorNames(input_pool, namespace=None):
  """"
  Printa input_pool.descriptorNames() de forma mais ordenada
  namespace é opcional. Se definido, printa apenas o namespace
  """
  assert(input_pool)
  printListNewline(input_pool.descriptorNames(namespace))

def duplicatePool(input_pool):
  """
  Retorna uma cópia profunda de input_pool
  """
  assert(input_pool)
  output_pool = es.Pool()
  output_pool.merge(input_pool)
  return output_pool


def aggregatePoolArraysToNumbers(input_pool):
  """
  Retorna uma cópia de input_pool onde usou es_mode.PoolAggregator() para 
  calcular as estatísticas dos arrays extraídos do áudio e insereri-las como números
  """
  output_pool = duplicatePool(input_pool)
  pool_arrays = es.Pool()
  for feat in output_pool.descriptorNames(pk_array):
    pool_arrays.merge(feat, output_pool[feat])

  aggr = es_mode.PoolAggregator()(pool_arrays)

  for feat in aggr.descriptorNames():
    newName = feat.replace("array", "number")
    output_pool.mergeSingle(newName, aggr[feat])

  return output_pool


def trimPool(input_pool, namespaces=[pk_array, pk_auxiliar, pk_extracted]):
  """
  Retorna uma cópia de input_pool, com os namespaces removidos
  Por default, remove os seguintes:
  - pk_array
  - pk_auxiliar
  - pk_extracted
  """
  assert(input_pool)
  output_pool = duplicatePool(input_pool)
  for ns in namespaces:
    output_pool.removeNamespace(ns)
  return output_pool

def flattenPool(input_pool, name):
  """
  Sabendo que input_pool tem uma estrutura:
  --features
     \
      --number
        \
          --feature1
          --feature2
          --feature3
          --feature_composta1
            \
              --subfeature1
              --subfeature2
              --subfeature2
          --feature_composta2
            \
              --subfeature1
              --subfeature2
              --subfeature2
              .
              .
              .
  Remove a etapa 'number' da hierarquia e achata as features compostas, para ficar assim:
  --features
    \
      --feature1
      --feature2
      --feature3
      --feature_composta1_subfeature1
      --feature_composta1_subfeature2
      --feature_composta1_subfeature2
      --feature_composta2_subfeature1
      --feature_composta2_subfeature2
      --feature_composta2_subfeature2
      .
      .
      .
  """
  output_pool = es.Pool()
  for desc in input_pool.descriptorNames():
    final = name+"."+desc.split(".", 2)[2].replace(".","_")
    output_pool.set(final, input_pool[desc])
  return output_pool

def setTarget(input_pool, target):
  """
  Recebe input_pool e uma string de qual é o target, e retorna
   a mesma com o atributo target definido como a seguir:
   - -1 - ERRO
   - 0 - polegar
   - 1 - tapa
  """
  assert(input_pool)
  assert(target)
  assert(isinstance(target, str))

  output_pool = duplicatePool(input_pool)
  target_number = -1

  if "polegar" in target: target_number = 0
  elif "tapa" in target: target_number = 1

  output_pool.set(pk_target, target_number)
  return output_pool

def savePoolToJson(input_pool, file_path="test_save.json"):
  """
  Salva input_pool no .json definido por file_name, 
  se o arquivo já existir, sobreescreve-o.

  O default de file_name serve para criar um arquivo de teste
  """
  
  try:
    remove(file_path)
  except FileNotFoundError:
    pass
  
  es_mode.YamlOutput(filename=file_path,\
                    format="json",\
                    writeVersion=False,\
                    doubleCheck=True)(input_pool)
  
def loadJsonToPool(file_path="test_save.json"):
  return es_mode.YamlInput(filename=file_path, format="json")()
  

def seeJson(file_path="test_save.json"):
  """
  Printa o .json denotado por file_path
  O default de file_path serve para ler o arquivo de teste
  """

  json_object = json.load(open(file_path))
  json_formatted_str = json.dumps(json_object, indent=2)
  print(json_formatted_str)

def testPoolSave(input_pool):
  """
  Testa as funções de ajuste de Pool usando input_pool
  e seus caminhos de arquivo padrão 
  """

  assert(input_pool)

  saved_pool = duplicatePool(input_pool)
  saved_pool = aggregatePoolArraysToNumbers(saved_pool)
  saved_pool = trimPool(saved_pool)
  savePoolToJson(saved_pool)

  seeJson()

  loaded_pool = loadJsonToPool()

  print("Pool salva e pool carregada são iguais?",\
        loaded_pool.descriptorNames()\
        ==trimPool(aggregatePoolArraysToNumbers(input_pool)).descriptorNames())
  
  return loaded_pool

def extractFeatures(audio_data):
  """
  Recebe um vetor de reais representando um sinal de áudio, calcula suas 
  features, agrega-as em uma Pool() de essentia e retorna esta Pool
  """
  from numpy import ndarray
  assert(type(audio_data) is ndarray)
  assert("float" in str(audio_data.dtype))

  #Inicia Pool()
  output_pool = es.Pool()

  #Calcula espectro do sinal
  output_pool.set(pk_spectrum, es_mode.Spectrum()(audio_data))

  #Calcula EnergyBandRatio
  energy_band_ratio = es_mode.EnergyBandRatio()(output_pool[pk_spectrum])
  output_pool.set(pk_energy_band_ratio, energy_band_ratio)

  #Calcula MaxMagFreq
  max_mag_freq = es_mode.MaxMagFreq()(output_pool[pk_spectrum])
  output_pool.set(pk_max_mag_freq, max_mag_freq)

  #Calcula SpectralCentroidTime
  spectral_centroid_time = es_mode.SpectralCentroidTime()(audio_data)
  output_pool.set(pk_spectral_centroid_time, spectral_centroid_time)

  #Calcula SpectralComplexity
  spectral_complexity = es_mode.SpectralComplexity()(output_pool[pk_spectrum])
  output_pool.set(pk_spectral_complexity, spectral_complexity)

  #Calcula StrongPeak
  strong_peak = es_mode.StrongPeak()(output_pool[pk_spectrum])
  output_pool.set(pk_strong_peak, strong_peak)

  #Calcula SpectralPeaks
  sp_freq, sp_mag = es_mode.SpectralPeaks()(output_pool[pk_spectrum])
  #corta o DC, se houver, e pedido de HarmonicPeaks
  if sp_freq[0]==0:
    sp_freq = sp_freq[1:]
    sp_mag = sp_mag[1:]
  output_pool.set(pk_spectral_peaks_freq, sp_freq)
  output_pool.set(pk_spectral_peaks_mag, sp_mag)


  ######################################
  #       Para Inharmonicity           #
  ######################################
  #Calcula PitchYinFFT
  pitch_yin_fft, pitch_prob_yin_fft = es_mode.PitchYinFFT()(output_pool[pk_spectrum])
  output_pool.set(pk_pitch, pitch_yin_fft)
  output_pool.set(pk_pitch_prob, pitch_prob_yin_fft)

  #Calcula HarmonicPeaks
  hp_freq, hp_mag = es_mode.HarmonicPeaks()(output_pool[pk_spectral_peaks_freq],\
                                            output_pool[pk_spectral_peaks_mag],\
                                            output_pool[pk_pitch] )
  output_pool.set(pk_harmonic_peaks_freq, hp_freq)
  output_pool.set(pk_harmonic_peaks_mag, hp_mag)

  #Calcula Inharmonicity
  inharmonicity = es_mode.Inharmonicity()(output_pool[pk_harmonic_peaks_freq],\
                                          output_pool[pk_harmonic_peaks_mag])
  output_pool.set(pk_inharmonicity, inharmonicity)

  #Acaba Inharmonicity#####################################

  #Calcula SpectralContrast
  frame_size = 2*(output_pool[pk_spectrum].size-1)
  spectral_contrast, spectral_valley = \
      es_mode.SpectralContrast(frameSize=frame_size)(output_pool[pk_spectrum])
  output_pool.set(pk_spectral_contrast, spectral_contrast)
  output_pool.set(pk_spectral_valley, spectral_valley)

  #Calcula SpectralWhitening
  spectral_whitening = \
              es_mode.SpectralWhitening()(output_pool[pk_spectrum],\
                                          output_pool[pk_spectral_peaks_freq],\
                                          output_pool[pk_spectral_peaks_mag])
  output_pool.set(pk_spectral_whitening, spectral_whitening)

  return output_pool

def computeAndSaveAudioFeatures(audio_data, name, file_path=None):
  """
  Computa e salva como json as features do áudio passado. Se file_path=None, 
  salva no arquivo de testes de json.
  """
  output_pool = extractFeatures(audio_data)
  output_pool = aggregatePoolArraysToNumbers(output_pool)
  output_pool = trimPool(output_pool)
  output_pool = setTarget(output_pool, name)
  output_pool = flattenPool(output_pool, name)
  savePoolToJson(input_pool=output_pool, file_path=file_path)